In [44]:
!pip install ucimlrepo

In [45]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [103]:
import warnings

warnings.filterwarnings('ignore')

In [104]:
from ucimlrepo import fetch_ucirepo 
import pandas as pd
  
# fetch dataset 
adult = fetch_ucirepo(id=2) 
  
# data (as pandas dataframes) 
X = adult.data.features 
y = adult.data.targets 
y['income'] = y['income'].str.replace(r'\.$', '', regex=True)

  
# metadata 
print(adult.metadata) 
  
# variable information 
print(adult.variables) 


{'uci_id': 2, 'name': 'Adult', 'repository_url': 'https://archive.ics.uci.edu/dataset/2/adult', 'data_url': 'https://archive.ics.uci.edu/static/public/2/data.csv', 'abstract': 'Predict whether annual income of an individual exceeds $50K/yr based on census data. Also known as "Census Income" dataset. ', 'area': 'Social Science', 'tasks': ['Classification'], 'characteristics': ['Multivariate'], 'num_instances': 48842, 'num_features': 14, 'feature_types': ['Categorical', 'Integer'], 'demographics': ['Age', 'Income', 'Education Level', 'Other', 'Race', 'Sex'], 'target_col': ['income'], 'index_col': None, 'has_missing_values': 'yes', 'missing_values_symbol': 'NaN', 'year_of_dataset_creation': 1996, 'last_updated': 'Tue Sep 24 2024', 'dataset_doi': '10.24432/C5XW20', 'creators': ['Barry Becker', 'Ronny Kohavi'], 'intro_paper': None, 'additional_info': {'summary': "Extraction was done by Barry Becker from the 1994 Census database.  A set of reasonably clean records was extracted using the fol

In [105]:
X = X.dropna()
y = y.loc[X.index]

In [106]:
from sklearn.preprocessing import LabelEncoder

X_encoded = pd.get_dummies(X, columns = ['workclass', 'education', 'marital-status', 'occupation', 'relationship', 'race', 'sex', 'native-country'])

le = LabelEncoder()
y['income'] = le.fit_transform(y['income'])

In [115]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report



n = len(X_encoded)//2

#Dividim el dataset
target_X = X_encoded.iloc[:n]
target_y = y.iloc[:n]

X_train, X_test, y_train, y_test = train_test_split(target_X, target_y, test_size=0.5)

target_out = list(X_test.index)

target_model = RandomForestClassifier(n_estimators=100)
target_model.fit(X_train, y_train)

preds_target = target_model.predict_proba(target_X)
target_df = pd.DataFrame({
        'prediction': preds_target.tolist(),
        'in_or_out': [1] * len(target_X)
    })
target_df.loc[target_df.index.isin(target_out), 'in_or_out'] = 0

target_df.dropna()
target_df[['pred_1', 'pred_2']] = pd.DataFrame(target_df['prediction'].tolist(), index=target_df.index)
target_df = target_df.drop('prediction', axis = 1)


# Predicciones sobre el conjunto de prueba
y_pred = target_model.predict(X_test)

# Precisión en el conjunto de prueba
accuracy = accuracy_score(y_test, y_pred)
print("\nPrecisión del modelo Random Forest en el conjunto de prueba:", accuracy)

# Reporte de clasificación en el conjunto de prueba
print("\nReporte de clasificación en el conjunto de prueba:")
print(classification_report(y_test, y_pred))

# Predicciones sobre el conjunto de entrenamiento
y_train_pred = target_model.predict(X_train)

# Precisión en el conjunto de entrenamiento
train_accuracy = accuracy_score(y_train, y_train_pred)
print("\nPrecisión del modelo Random Forest en el conjunto de entrenamiento:", train_accuracy)

# Reporte de clasificación en el conjunto de entrenamiento
print("\nReporte de clasificación en el conjunto de entrenamiento:")
print(classification_report(y_train, y_train_pred))


Precisión del modelo Random Forest en el conjunto de prueba: 0.854514909701806

Reporte de clasificación en el conjunto de prueba:
              precision    recall  f1-score   support

           0       0.89      0.93      0.91      9041
           1       0.73      0.62      0.67      2864

    accuracy                           0.85     11905
   macro avg       0.81      0.78      0.79     11905
weighted avg       0.85      0.85      0.85     11905


Precisión del modelo Random Forest en el conjunto de entrenamiento: 1.0

Reporte de clasificación en el conjunto de entrenamiento:
              precision    recall  f1-score   support

           0       1.00      1.00      1.00      9096
           1       1.00      1.00      1.00      2809

    accuracy                           1.00     11905
   macro avg       1.00      1.00      1.00     11905
weighted avg       1.00      1.00      1.00     11905



In [112]:
X_shadow = X_encoded.iloc[n:]
y_shadow = y.iloc[n:]
X_train, X_test, y_train, y_test = train_test_split(target_X, target_y, test_size=0.5)

shadow_out = list(X_test.index)

model_rf = RandomForestClassifier(n_estimators=100)
model_rf.fit(X_train, y_train)

preds = model_rf.predict_proba(X_shadow)


result_df = pd.DataFrame({
        'prediction': preds.tolist(),
        'in_or_out': [1] * len(X_shadow)
    })

result_df.loc[result_df.index.isin(shadow_out), 'in_or_out'] = 0

result_df.dropna()
result_df[['pred_1', 'pred_2']] = pd.DataFrame(result_df['prediction'].tolist(), index=result_df.index)
result_df = result_df.drop('prediction', axis = 1)

In [109]:
final_X = result_df.drop('in_or_out', axis = 1)
final_y = result_df['in_or_out'].astype(int)


X_train, X_test, y_train, y_test = train_test_split(final_X, final_y, test_size=0.25)

print(X_train.shape)

attack_model = RandomForestClassifier(n_estimators=100)
attack_model.fit(X_train, y_train)

y_pred = attack_model.predict(X_test)

accuracy = accuracy_score(y_test, y_pred)
print("\nPrecisión del modelo Random Forest en el conjunto de prueba:", accuracy)

# Reporte de clasificación en el conjunto de prueba
print("\nReporte de clasificación en el conjunto de prueba:")
print(classification_report(y_test, y_pred))

# Predicciones sobre el conjunto de entrenamiento
y_train_pred = attack_model.predict(X_train)

# Precisión en el conjunto de entrenamiento
train_accuracy = accuracy_score(y_train, y_train_pred)
print("\nPrecisión del modelo Random Forest en el conjunto de entrenamiento:", train_accuracy)

# Reporte de clasificación en el conjunto de entrenamiento
print("\nReporte de clasificación en el conjunto de entrenamiento:")
print(classification_report(y_train, y_train_pred))

(26787, 2)

Precisión del modelo Random Forest en el conjunto de prueba: 0.839511703438235

Reporte de clasificación en el conjunto de prueba:
              precision    recall  f1-score   support

           0       0.00      0.00      0.00      1433
           1       0.84      1.00      0.91      7496

    accuracy                           0.84      8929
   macro avg       0.42      0.50      0.46      8929
weighted avg       0.70      0.84      0.77      8929


Precisión del modelo Random Forest en el conjunto de entrenamiento: 0.8312614327845597

Reporte de clasificación en el conjunto de entrenamiento:
              precision    recall  f1-score   support

           0       0.00      0.00      0.00      4520
           1       0.83      1.00      0.91     22267

    accuracy                           0.83     26787
   macro avg       0.42      0.50      0.45     26787
weighted avg       0.69      0.83      0.75     26787



In [114]:
X_attacked = target_df.drop('in_or_out', axis = 1)
y_attacked = target_df['in_or_out']

y_pred = attack_model.predict(X_attacked)

accuracy = accuracy_score(y_attacked, y_pred)

print(classification_report(y_attacked, y_pred))

              precision    recall  f1-score   support

           0       0.00      0.00      0.00      5953
           1       0.50      1.00      0.67      5952

    accuracy                           0.50     11905
   macro avg       0.25      0.50      0.33     11905
weighted avg       0.25      0.50      0.33     11905

